In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw03.ipynb")

<div class="alert alert-success">

#### Homework 3 Supplemental Notebook
    
# Vectors and the Dot Product

### EECS 245, Winter 2026 at the University of Michigan
    
</div>

### Instructions

Most homeworks will have Jupyter Notebooks, like this one, designed to supplement the theoretical problems. 

To write and run code in this notebook, you have two options:

1. **Set up a Jupyter Notebook environment locally, and use `git` to clone our course repository (preferred).** For instructions on how to do this, see the [**Environment Setup**](https://eecs245.org/env-setup) page of the course website.
1. **Use the EECS 245 DataHub.** To do this, click the link provided in the Homework 3 PDF. Before doing so, read the instructions on the [**Environment Setup**](https://eecs245.org/env-setup/#option-2-using-the-eecs-245-datahub) page on how to use the DataHub.

To receive credit for the programming portion of the homework, you'll need to submit your completed notebook to the autograder on Gradescope. Your submission time for Homework 3 is the **latter** of your PDF and code submission times.

<div class="alert alert-info">
    Since this problem is brand-new this semester, there are no hidden test cases, just in case there are bugs in the code. The tests in your notebook are the same as the test cases on Gradescope, and when you submit to Gradescope, you should be able to see your final score.
</div>

In [ ]:
# Run this cell.
import numpy as np
import pandas as pd
import util
from IPython.display import HTML

## Problem 6: Neighbors 🏡 (10 pts)

---

So far in EECS 245, we've focused on **regression** problems, where the goal is to predict a continuous value, e.g. commute time. In this problem, we'll explore a **classification** problem, where the goal is to predict a categorical value. Examples of classification problems include:
- Does this person have diabetes?
- Is this digit a 0, 1, 2, 3, 4, 5, 6, 7, 8, or 9? (We saw this in [Chapter 1.1](https://notes.eecs245.org/introduction-to-supervised-learning/what-is-machine-learning/)).
- Is this picture of a dog, cat, zebra, or hamster?

We'll explore the first problem here: predicting whether or not a patient has diabetes, given their `Glucose` and `BMI` levels. Let's start by loading the data.

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')
diabetes

`Glucose` is measured in mg/dL (milligrams per deciliter); `BMI` is calculated as $\text{BMI} = \frac{\text{weight (kg)}}{\left[ \text{height (m)} \right]^2}$. In the `Outcome` column, `0` means no diabetes, `1` means yes diabetes.

In [ ]:
# 64.9% of individuals in the dataset do not have diabetes.
diabetes['Outcome'].value_counts(normalize=True)

Before we visualize the data, we'll split the dataset into training and test sets. The training set will be used to fit the model, and the test set will be used to evaluate the model on "unseen" data. This is an idea we'll revisit in future homeworks.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = (
    train_test_split(diabetes[['Glucose', 'BMI']], diabetes['Outcome'], random_state=1)
)

X_train

Above, `X_train` is a DataFrame (technically not a `numpy` array) containing a random sample of 75% of the rows from the full dataset, and just the `Glucose` and `BMI` columns. We will not work directly with DataFrames: instead, the functions you implement below will take in `numpy` arrays. We'll handle the DataFrame-to-array conversion for you once it's necessary.

Below, <span style='color: orange'><b>class 0 (orange) is "no diabetes"</b></span> and <span style='color: blue'><b>class 1 (blue) is "diabetes"</b></span>.

In [ ]:
fig = util.create_base_scatter(X_train, y_train)
fig

Using this dataset, how can we classify whether someone new (not already in the dataset) has diabetes, given their `Glucose` and `BMI`?

**Here's the intuition behind the $k$-nearest neighbors 🏡 classifier**: if a new person's feature vector is <b><span style="color:blue">close to the blue points</span></b>, we'll predict <b><span style="color:blue">blue (diabetes)</span></b>; if they're <b><span style="color:orange">close to the orange points</span></b>, we'll predict <b><span style="color:orange">orange (no diabetes)</span></b>.

That is, suppose we're given a new individual, $\vec{x}_\text{new} = \begin{bmatrix} \text{Glucose}_\text{new} \\ \text{BMI}_\text{new} \end{bmatrix}$. The $k$-nearest neighbors classifier ($k$-NN for short) classifies $\vec{x}_\text{new}$ by:
1. Finding the $k$ **closest points** in the training set to $\vec{x}_\text{new}$.
1. Predicting that $\vec{x}_\text{new}$ belongs to the **most common class** among those $k$ closest points.

For example, suppose $k = 6$. If, among the 6 closest points to $\vec{x}_\text{new}$, there are <b><span style="color:blue">4 blue</span></b> and <b><span style="color:orange">2 orange</span></b> points, we'd predict <b><span style="color:blue">blue (diabetes)</span></b>. If there are ties, we'll have to break them somehow; read [here](https://stats.stackexchange.com/questions/144718/how-does-scikit-learn-resolve-ties-in-the-knn-classification) for more information.

Some questions you may have:
- How do we measure "closeness" between two points? Suppose $\vec x_i$ and $\vec x_j$ are two vectors in $\mathbb{R}^n$. To measure how close they are, we can compute the **norm (i.e. length) of the difference between the two vectors**. (Refer to [Chapter 3.2](https://notes.eecs245.org/vectors/norms/) for a discussion of norms.) There are multiple vector norms, which means we have a choice to make:
    $$\text{using the $L_2$ norm: } \text{dist}(\vec x_i, \vec x_j) = \|\vec x_i - \vec x_j\|_2 = \sqrt{\sum_{d=1}^n (\text{component $d$ of } \vec x_i - \text{component $d$ of } \vec x_j)^2}$$
    $$\text{using the $L_1$ norm: } \text{dist}(\vec x_i, \vec x_j) = \|\vec x_i - \vec x_j\|_1 = \sum_{d=1}^n |\text{component $d$ of } \vec x_i - \text{component $d$ of } \vec x_j|$$
    For instance, if $\vec x_1 = \begin{bmatrix} 1 \\ 2 \end{bmatrix}$ and $\vec x_2 = \begin{bmatrix} 4 \\ 6 \end{bmatrix}$, then $\|\vec x_1 - \vec x_2\|_2 = 5$ and $\|\vec x_1 - \vec x_2\|_1 = 7$.
- What value of $k$ should we use, i.e., how many neighbors should we look at? The value of $k$ is called a **hyperparameter**, because it's a parameter of the model that we get to choose (unlike, say, the $w_0$'s and $w_1$'s in simple linear regression, which are found through minimizing mean squared error, not by us picking them). Soon, we'll see how different values of $k$ can lead to different models.

This problem will walk you through implementing the $k$-nearest neighbors classifier. By the end, you'll implement a `KNNClassifier` class (similar to the `SimpleLAD` class from Homework 2) that you can use to classify new points.

### Problem 6a) (2 pts)

Complete the implementation of `lp_distance`, which takes in:
- `x_i` and `x_j`, two 1D `numpy` arrays with the same number of components, and
- `p`, an integer equal to either `1` or `2`, corresponding to the $L_1$ or $L_2$ norm.

`lp_distance` should return a **float** representing $\|\vec x_i-\vec x_j\|_p$. That is, if `p = 1`, it should return the $L_1$ norm of $\vec x_i - \vec x_j$; if `p = 2`, it should return the $L_2$ norm of $\vec x_i - \vec x_j$. Example behavior is given below.

```python
>>> lp_distance(np.array([1, 2]), np.array([4, 6]), 1)
7.0

>>> lp_distance(np.array([1, 2]), np.array([4, 6]), 2)
5.0
```

**Don't** use a `for`-loop, and also don't use `np.linalg.norm`. Instead, use `np.abs`, `np.sum`, etc.

In [ ]:
def lp_distance(x_i, x_j, p):
    if p not in [1, 2]:
        raise ValueError('p must be 1 or 2')
        
    if x_i.shape != x_j.shape:
        raise ValueError('x_i and x_j must have the same shape')

    ...

# Feel free to change these inputs to test your function.
print(lp_distance(np.array([1, 2]), np.array([4, 6]), 1))
print(lp_distance(np.array([1, 2]), np.array([4, 6]), 2))

In [ ]:
grader.check("p06_a")

### Problem 6b) (2 pts)

Complete the implementation of `all_distances`, which takes in:
- `X_train`, a 2D `numpy` array with shape `(n, d)`,
- `x_new`, a 1D `numpy` array with `d` components, and
- `p`, an integer equal to either `1` or `2`, corresponding to the $L_1$ or $L_2$ norm.

`all_distances` should return a 1D `numpy` array of length `n` containing distances from `x_new` to each row of `X_train`. Example behavior is given below.

```python
>>> X_demo = np.array([
    [1, 2],
    [2, 3],
    [3, 4],
    [6, 10]
])
>>> all_distances(X_demo, np.array([1, 1]), 1)
array([1.0, 3.0, 5.0, 14.0])

>>> all_distances(X_demo, np.array([1, 1]), 2)
array([1.0, 2.24, 3.61, 10.3])
```

You **can** use a `for`-loop, though this can be done without one. To access row `i` of a `numpy` array `X`, you can use `X[i]`. Column `j` (if you need it) can be accessed with `X[:, j]`.

In [ ]:
def all_distances(X_train, x_new, p):
    """Return a 1D array of distances from x_new to each row of X_train."""
    ...

# Feel free to change these inputs to test your function.
X_demo = np.array([
    [1, 2],
    [2, 3],
    [3, 4],
    [6, 10]
])
print(all_distances(X_demo, np.array([1, 1]), 1))
print(all_distances(X_demo, np.array([1, 1]), 2))

In [ ]:
grader.check("p06_b")

Great work so far. We're going to handle the next step for you. Below, we've implemented the function `knn_indices`, which takes in:
- `X_train`, a 2D `numpy` array with shape `(n, d)`,
- `x_new`, a 1D `numpy` array with `d` components,
- `k`, the number of nearest neighbors to find in the algorithm, and
- `p`, an integer equal to either `1` or `2`, corresponding to the $L_1$ or $L_2$ norm.

`knn_indices` returns a 1D array of length `k` (not `n` or `d`!) containing the integer indices of the `k` nearest neighbors of `x_new` in `X_train`. It does this by calling your `all_distances` function on `X_train` and `x_new`, and then using `np.argsort` to find the indices of the smallest `k` values in the distance array. We use mergesort's handling of ties.

In [ ]:
def knn_indices(X_train, x_new, k, p):
    dists = all_distances(X_train, x_new, p)
    return np.argsort(dists, kind='mergesort')[:k]

Let's test it out.

In [ ]:
X_demo_shuffled = np.array([
    [3, 4],
    [2, 3],
    [6, 10],
    [1, 2]
])
knn_indices(X_demo_shuffled, np.array([1, 1]), k=3, p=2)

**If you've implemented everything correctly so far**, the above should show `np.array([3, 1, 0])`. This is because the 3rd row of `X_demo` is closest to `x_new` in the $L_2$ norm, the 1st row is the second closest, and the 0th row is the third closest.

<div class="alert alert-block alert-info">
    Make sure you understand how the call above works before moving forward!
</div>

### Problem 6c) (2 pts)

So far, we've only dealt with the input variables, `Glucose` and `BMI`. Now, let's stitch our work together to actually classify whether someone has diabetes.

Complete the implementation of `knn_predict`, which takes in:
- `X_train`, a 2D `numpy` array with shape `(n, d)`,
- `y_train`, a 1D `numpy` array with `d` components, containing the values `1` (yes diabetes) or `0` (no diabetes),
- `x_new`, a 1D `numpy` array with `d` components,
- `k`, the number of nearest neighbors to find in the algorithm, and
- `p`, an integer equal to either `1` or `2`, corresponding to the $L_1$ or $L_2$ norm.

`knn_predict` should return a `1` or `0` representing the predicted class label for `x_new`, by finding the most common class (1 or 0) among the `k` nearest neighbors of `x_new` in `X_train`. Use the `knn_indices` function you implemented above to find the neighbors. If `k` is even and there are an equal number of 1s and 0s among the nearest neighbors, you should return `1`.

Example behavior is given below.

```python
>>> X_demo = np.array([
    [1, 2],
    [2, 3],
    [3, 4],
    [6, 10]
])
>>> y_demo = np.array([0, 0, 1, 1])
>>> knn_predict(X_demo, y_demo, np.array([1, 1]), k=3, p=2)
0 # Make sure you understand where this came from!
```

**Don't** use a `for`-loop. Once you have the indices of the nearest neighbors, you can use them to index into `y_train` to get the labels of the nearest neighbors:

```python
>>> idx = np.array([3, 5, 1])
>>> y[idx] # This is valid syntax, and will return just the elements of y at positions 3, 5, and 1.
```

In [ ]:
def knn_predict(X_train, y_train, x_new, k, p):
    ...

# Feel free to change these inputs to test your function.
# In particular, test it on values of k that are larger than 2 or 3!
X_demo = np.array([
    [1, 2],
    [2, 3],
    [3, 4],
    [6, 10]
])
y_demo = np.array([0, 0, 1, 1])
knn_predict(X_demo, y_demo, np.array([1, 1]), k=3, p=2)

In [ ]:
grader.check("p06_c")

### Problem 6d) (2 pts)

Finally, complete the implementation of the `KNNClassifier` class, which has two methods, apart from the constructor.

#### `fit`

All `fit` should do is store the training data in the instance variables `self.X_train_` and `self.y_train_`. Unlike `SimpleLAD` in Homework 2, there are no parameters to solve for, so `fit` doesn't really need to do anything else. ($k$-NN is called a **non-parametric method**.)

#### predict

`predict` takes in a single (non-`self`) input, named `x_new`.
- If `x_new` is a 1D array, `predict` should return a single prediction (either `1` or `0`) as an integer.
- If `x_new` is a 2D array, `predict` should return a 1D array of predictions, one for each row in `x_new`. **You can `for`-loop in this case.**

Example behavior is given below.

```python
>>> X_small = np.array([
    [148, 33.6],
    [85, 26.6],
    [183, 23.3],
    [89, 28.1],
    [137, 43.1],
    [116, 25.6],
])
>>> y_small = np.array([1, 0, 0, 1, 1, 0])
>>> model = KNNClassifier(k=5, p=2)
>>> model.fit(X_small, y_small) # No output is expected!

>>> x_new = np.array([110, 30.0])
>>> model.predict(x_new)
0

>>> x_new_multiple = np.array([
    [110, 30.0],
    [148, 33.6],
    [85, 26.6],
])
>>> model.predict(x_new_multiple)
array([0, 1, 0])
```

In [ ]:
class KNNClassifier:
    def __init__(self, k=3, p=2):
        self.k = k
        self.p = p

    def fit(self, X_train, y_train):
        if len(X_train) != len(y_train):
            raise ValueError(f'Dimension mismatch: X has length {len(X_train)} while y has length {len(y_train)}')

        if np.unique(y_train).size != 2:
            raise ValueError('y_train must contain exactly two classes.')

        # Your solution only needs to be two lines of code.
        # They are very short; we're leaving them as a task for you just to make sure you understand the code.
        ...

    # Hint: You'll find yourself using many instance variables, like
    # self.X_train_ and self.p, in your implementation.
    def predict(self, x_new):
        if isinstance(x_new, list):
            x_new = np.array(x_new)
        try:
            # If x_new is a single point:
            if x_new.ndim == 1:
                ...
            # Otherwise, x_new is a 2D array:
            else:
                ...
        except AttributeError:
            raise AttributeError('Cannot use `predict` before `fit`.')

# Feel free to change these inputs to test your class implementation.
X_small = np.array([
    [148, 33.6],
    [85, 26.6],
    [183, 23.3],
    [89, 28.1],
    [137, 43.1],
    [116, 25.6],
])
y_small = np.array([1, 0, 0, 1, 1, 0])

model = KNNClassifier(k=1, p=2)
model.fit(X_small, y_small)

x_new = np.array([110, 30.0])
print(model.predict(x_new))

x_new_multiple = np.array([
    [110, 30.0],
    [148, 33.6],
    [85, 26.6],
])

print(model.predict(x_new_multiple))

In [ ]:
grader.check("p06_d")

Awesome work! Now, it's time to enjoy the fruits of your labor. Like we did at the start of the notebook, we'll load in the full dataset and divide it into training and test sets. (We're redoing this here in case we overrode the original `X_train` and `y_train` variables.)

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')
X_train, X_test, y_train, y_test = (
    train_test_split(diabetes[['Glucose', 'BMI']], diabetes['Outcome'], random_state=1)
)
X_train, y_train = X_train.to_numpy(), y_train.to_numpy()
X_test, y_test = X_test.to_numpy(), y_test.to_numpy()
X_train

Now, we'll fit an instance on **just** the training data. For now, we'll arbitrarily use `k=5` and `p=2`.

In [ ]:
model = KNNClassifier(k=5, p=2)
model.fit(X_train, y_train)

One of the first metrics we'll check when training a classifier on a training set is its **training set accuracy** and **test set accuracy**. The accuracy of a model is the fraction of predictions that match the true labels.

In [ ]:
model.predict(X_train)

In [ ]:
# For each point in the training set, check if the model's prediction matches the true label.
model.predict(X_train) == y_train

In [ ]:
# Training set accuracy
(model.predict(X_train) == y_train).mean()

In [ ]:
# Test set accuracy
(model.predict(X_test) == y_test).mean()

So, our model correctly classifies 81.5% of the training data and 74.4% of the test data.

What does it **look like**? Let's plot the **decision boundary** of our model. Run the cell below; it may take a few seconds to appear.

In [ ]:
util.show_decision_boundary(model, X_train, y_train, title='k-NN Decision Boundary (k=5, p=2)')

Regions colored <span style="color:blue"><b>blue</b></span> are regions where the model predicts a label of 1 (**diabetes**), and regions colored <span style="color:orange"><b>orange</b></span> are regions where the model predicts a label of 0 (**no diabetes**). So, if some new patient $\vec x_\text{new} = \begin{bmatrix} \text{Glucose}_\text{new} \\ \text{BMI}_\text{new} \end{bmatrix}$ comes along, all we need to do is check which region it falls into and we've made our prediction! The points shown come from the training set; the test set is not visualized at all.

The decision boundary we ended up with depends on our choices of `k` (number of neighbors) and `p` (distance metric). Let's see how these choices affect the decision boundary. First, let's try changing `p` to `1` and looking at how the decision boundary changes.

In [ ]:
model_p1 = KNNClassifier(k=5, p=1)
model_p1.fit(X_train, y_train)
util.show_decision_boundary(model_p1, X_train, y_train, title='k-NN Decision Boundary (k=5, p=1)')

The decision boundary mostly looks the same as when `p=2`, but there are some subtle differences, particularly in the top right. Think about why this is.

A more pronounced difference is what happens when we change `k`. Below, you'll find a slider that allows you to change the value of `k` and see the effect on the decision boundary.

In [ ]:
with open('imgs/knn_slider.html') as f:
    display(HTML(f.read()))

Above, you should notice that as the value of `k` increases, the decision boundary becomes more smooth and less jagged. Very small values of `k` result in the model **overfitting** to the training data. After all, the closest neighbor to a point in the training set is the point itself, so using a low value of `k` results in the model effectively memorizing the training data. This is unlikely to generalize well to the test set. On the other hand, very large values of `k` result in a very simple model that may not be sophisticated enough to capture the complexity of the data.

Which value of `k` should we use, then? One idea is to test out all 50 of the models above – `k=1`, `k=2`, ..., `k=50` – on the test set, picking the value of `k` with the **highest test set accuracy**. Let's do this below.

### Problem 6e) (2 pts)

In the space provided, train 50 models, one for each value of `k` from `k=1` to `k=50`. 
- Each one should be trained on `X_train` and `y_train` with `p=2`.
- For each model, calculate the **test set accuracy** of the model. We did an example for you above.
- Finally, assign `best_k` to the value of `k` with the **highest test set accuracy**.

All we are autograding is the value of `best_k`, but you'll need to write a few lines of code to find it. Don't manually set the value of `best_k`: once you've created a list/array of test accuracies, use `np.argmax` to find the value of `k` with the highest accuracy (and make sure to add 1 to the result, since `np.argmax` returns 0-based indices). It turns out that there are multiple values of `k` that result in the same highest test set accuracy; you just need to find any one of them.

In [ ]:
best_k

In [ ]:
grader.check("p06_e")

Think about what would happen if we let `k` get even larger, like `k=100` or `k=200`! (How does this relate to the constant model from the first week of the semester?)

## Finish Line 🏁

Congratulations! You're ready to submit the programming portion of Homework 3.

To submit your work to Gradescope:

1. Select `Kernel -> Restart & Run All` to ensure that you have executed all cells, including the test cells.
2. Read through the notebook to make sure everything is fine and all public tests passed.
3. Run the cell below to run all tests, and make sure that they all pass.
4. Download your notebook using `File -> Download`, then upload your notebook to Gradescope under "Homework 3, Problem 6 Code".
5. Stick around for a few minutes while the Gradescope autograder grades your work. **Remember that this homework has no hidden tests, so you should be able to see your score shortly after submitting.**